In [ ]:
import logging
import sys

import pandas as pd

from hamilton import base, driver
from hamilton.io.materialization import to

logging.basicConfig(stream=sys.stdout)

In [ ]:
# We use the autoreload extension that comes with ipython to automatically reload modules when
# the code in them changes.

# import the jupyter extension
%load_ext autoreload
# set it to only reload the modules imported
%autoreload 1
# import the function modules you want to reload when they change.
# i.e. these should be your modules you write your functions in. As you change them,
# they will be reimported without you having to do anything.
%aimport my_functions

In [ ]:
#   Define your new Hamilton functions & curate them into a TemporaryFunctionModule object.
# This enables you to add functions to your DAG without creating a proper module.

# Look at `my_functions` to see how these functions connect.
def signups() -> pd.Series:
    """Returns sign up values"""
    return pd.Series([1, 10, 50, 100, 200, 400])


def spend() -> pd.Series:
    """Returns the spend values"""
    return pd.Series([10, 10, 20, 40, 40, 50])


def log_spend_per_signup(spend_per_signup: pd.Series) -> pd.Series:
    """Simple function taking the logarithm of spend over signups."""
    return np.log(spend_per_signup)


# Place the functions into a temporary module -- the idea is that this should house a curated set of functions.

temp_module = ad_hoc_utils.create_temporary_module(
    spend, signups, log_spend_per_signup, module_name="function_example"
)

In [ ]:
initial_columns = {  # load from actuals or wherever -- this is our initial data we use as input.
    # Note: these values don't have to be all series, they could be a scalar.
    "signups": pd.Series([1, 10, 50, 100, 200, 400]),
    "spend": pd.Series([10, 10, 20, 40, 40, 50]),
}
# we need to tell hamilton where to load function definitions from
# programmatic code to load modules:
# module_name = "my_functions"
# my_functions = importlib.import_module(module_name)
# or import module(s) directly:

import my_functions

In [ ]:
df_builder = base.PandasDataFrameResult()

dr = driver.Driver({}, my_functions)  # can pass in multiple modules

In [ ]:
# we need to specify what we want in the final dataframe. These can be string names, or function references.
output_columns = [
    "spend",
    "signups",
    "avg_3wk_spend",  # could just pass "avg_3wk_spend" here
    "spend_per_signup",  # could just pass "spend_per_signup" here
    "spend_zero_mean_unit_variance",  # could just pass "spend_zero_mean_unit_variance" here
]

In [ ]:
materializers = [
    # materialize the dataframe to a pickle file
    to.pickle(
        dependencies=output_columns,
        id="df_to_pickle",
        path="./df.pkl",
        combine=df_builder,
    ),
]
# Visualize what is happening
dr.visualize_materialization(
    *materializers,
    additional_vars=output_columns,
    output_file_path="./dag",
    render_kwargs={},
    inputs=initial_columns,
)

In [ ]:
# Materialize a result, i.e. execute the DAG!
materialization_results, additional_outputs = dr.materialize(
    *materializers,
    additional_vars=[
        "df_to_pickle_build_result"
    ],  # because combine is used, we can get that result here.
    inputs=initial_columns,
)

In [ ]:
print(materialization_results)


In [ ]:
print(additional_outputs["df_to_pickle_build_result"])